In [1]:
import os
import pandas as pd
from IPython.display import display_html, HTML

In [2]:
prefixes_path = "../server/embeddings/analysis/prefixes.txt"
data_path = "../data"

In [3]:
with open(prefixes_path, "r") as f:
    prefixes = [line.rstrip() for line in f.readlines()]

In [4]:
index_set_single = [file.split(".")[0] for file in os.listdir(os.path.join(data_path, "indexed/single_chart"))]
index_set_multiple = [file.split(".")[0] for file in os.listdir(os.path.join(data_path, "indexed/multiple_chart/imgs"))]

index_set = []
index_set.extend(index_set_single)
index_set.extend(index_set_multiple)
index_set.sort()

len(index_set)

161

In [5]:
all_data = [file.split(".")[0] for file in os.listdir(os.path.join(data_path, "unified/specs"))]
all_data.sort()

In [6]:
df = pd.DataFrame(all_data, columns=["file_name"])

df['class'] = df['file_name'].apply(lambda x: next((pre for pre in prefixes if x.startswith(pre)), None))
df['index'] = df['file_name'].apply(lambda x: True if x in index_set else False)

In [7]:
df_index = df[df['index']]
index_dict = df_index.set_index('file_name')['class'].to_dict()

In [8]:
chromoscope = ["BOCA", "BRCA", "BTCA", "GBM", "LIRI", "PBCA"]
chromoscope2 = ["breast-", "gastric", "kidney", "ovarian serous cystadenocarcinoma", "ovarian", "prostate adenocarcinoma", "sarcoma"]
missed = [] #temp

def get_original(file_name, prefix):
    if file_name in index_set:
        return file_name
    if prefix.split("-")[0] in chromoscope or prefix.split("-")[0] in chromoscope2:
        return 'BOCA-UK-f83fc777-5416-c3e9-e040-11ac0d482c8e'
    if "_oc" or "_cc_" in file_name:
        matches = [index for index, index_pre in index_dict.items() if index_pre == prefix]
        if len(matches) == 0: 
            # print("error on no matches at all?", file_name)
            missed.append(file_name)
            return None
        elif len(matches) == 1: 
            return matches[0]
        
        # this is only for the EX_SPEC_CYTOBANDS example
        if file_name.endswith('circular'):
            return matches[1]
        if file_name.endswith('serial'):
            return None
        
        if "_oc" in file_name:
            filtered_matches = [match for match in matches if match.endswith("_oc")]
            if len(filtered_matches) == 0: 
                print("error on _oc?", file_name)
            return filtered_matches[0]
        else:
            color = file_name.split("_cc_")[-1]
            filtered_matches = [match for match in matches if f"_cc_{color}" in match] 
            if len(filtered_matches) == 0:
                return matches[0]
            return filtered_matches[0]
    missed.append(file_name)
    return None

In [9]:
df['original'] = df.apply(lambda row: get_original(row['file_name'], row['class']), axis=1)

In [10]:
len(missed)

706

In [11]:
df.head()

,file_name,class,index,original
0,AREA_sw_0_7_s_0_7_cc_0,AREA,True,AREA_sw_0_7_s_0_7_cc_0
1,AREA_sw_0_7_s_0_7_cc_1,AREA,False,AREA_sw_0_7_s_1_0_cc_1
2,AREA_sw_0_7_s_0_7_cc_2,AREA,False,AREA_sw_1_2_s_0_7_cc_2
3,AREA_sw_0_7_s_0_7_cc_3,AREA,False,AREA_sw_0_7_s_1_2_cc_3
4,AREA_sw_0_7_s_0_7_oc,AREA,False,AREA_sw_1_0_s_0_7_oc


In [12]:
with open("missed.txt", "w") as f:
    for m in missed: 
        f.write(m)
        f.write("\n")

In [13]:
df.to_csv("siblings.csv")

In [14]:
result = df.groupby('original')['file_name'].apply(list).to_dict()
result = {key : [v for v in value if v != key] for key, value in result.items()}
import json

with open("siblings.json", "w") as f:
    f.write(json.dumps(result))

In [ ]:
df["image"] = df["file_name"].apply(lambda x: f"{data_path}/unified/imgs/{x}.png")
df["image_original"] = df["original"].apply(lambda x: f"{data_path}/unified/imgs/{x}.png")

def display_images_in_table(df):
    html = """<table border='1' style='border-collapse: collapse; text-align: left;'>
              <tr>"""  

    html += "".join(f"<th style='padding: 4px; text-align: left; white-space: normal;'>{col}</th>" for col in df.columns if (col != "image" and col != "image_original"))
    html += "<th style='padding: 4px;'>Image</th>"
    html += "<th style='padding: 4px;'>Image Original</th></tr>"

    for _, row in df.iterrows():
        html += "<tr>"
        html += "".join(f"<td style='padding: 4px; text-align: left; white-space: normal; word-break: break-word;'>{row[col]}</td>" for col in df.columns if (col != "image" and col != "image_original"))
        html += f"<td style='padding: 4px;'><img src='{row['image']}' style='width:300px;'></td>"
        html += f"<td style='padding: 4px;'><img src='{row['image_original']}' style='width:300px;'></td>"
        html += "</tr>"

    html += "</table>"

    display_html(HTML(html))
    
display_images_in_table(df)